# Tabular classification with Amazon SageMaker TabTransformer algorithm

## Load Libraries

In [33]:
import numpy as np
import pandas as pd
import os
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split
import sagemaker, boto3, json
from sagemaker import get_execution_role

## First, we store the training and validation data in S3 as instructed

In [34]:
# include all paths to data from local storage location
TRAIN_DATA = os.environ['DATAFILES_PATH'] + '/ICR_Competition/' + 'train.csv'
TEST_DATA = os.environ['DATAFILES_PATH'] + '/ICR_Competition/' + 'test.csv'
GREEKS_DATA = os.environ['DATAFILES_PATH'] + '/ICR_Competition/' + 'greeks.csv'

# load training data
train_df = pd.read_csv(TRAIN_DATA)

# allocate
X = train_df.drop(columns=['Class', 'Id'])
X = pd.get_dummies(X, drop_first=True)

y = train_df['Class'].astype(int)

# train-validation split 
X_train_raw, X_val, y_train_raw, y_val = train_test_split(X, y, test_size=0.20, random_state=42)

# over sample the diagnosed patients in training set
ros = RandomOverSampler(random_state=77, sampling_strategy='minority')
X_train, y_train = ros.fit_resample(X_train_raw, y_train_raw)

# shuffle (in case the model choice may be impacted by ordering)
shuff_ind = np.random.choice(len(y_train), len(y_train), replace=False)

X_train = X_train.iloc[shuff_ind,]
y_train = y_train.iloc[shuff_ind,]

# make uints just in case 
y_train.astype('uint8')
y_val.astype('uint8')

# create df for train and val as instructed by aws
df_train = pd.concat([y_train, X_train], axis=1)
numericalVar = df_train.columns.drop(['EJ_B'])
df_train[numericalVar] = df_train[numericalVar].fillna(df_train[numericalVar].mean())
df_train = df_train.dropna()

df_val = pd.concat([y_val, X_val], axis=1)
df_val[numericalVar] = df_val[numericalVar].fillna(df_train[numericalVar].mean())
df_val = df_val.dropna()

if not os.path.exists('train'):
    os.mkdir('train')

df_train.to_csv('train/data.csv', index=False, header=False)

if not os.path.exists('validation'):
    os.mkdir('validation')

df_val.to_csv('validation/data.csv', index=False, header=False)


# Upload the files to s3
s3 = boto3.resource('s3')
response = s3.meta.client.upload_file('train/data.csv', 
                                      'mypersonalprojectdata', 
                                      'ICR-Data/train/data.csv')

response = s3.meta.client.upload_file('validation/data.csv', 
                                      'mypersonalprojectdata', 
                                      'ICR-Data/validation/data.csv')


# remove files and directories locally
os.remove('train/data.csv')
os.remove('validation/data.csv')
os.rmdir('train')
os.rmdir('validation')

## Next, we set up SageMaker training session

In [35]:
sess = sagemaker.Session(boto3.session.Session(region_name='us-west-1'))
aws_region = boto3.Session().region_name
aws_role = 'arn:aws:iam::378421839225:role/service-role/AmazonSageMaker-ExecutionRole-20230623T185897'

INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials


In [36]:
from sagemaker import image_uris, model_uris, script_uris

train_model_id, train_model_version, train_scope = (
    "pytorch-tabtransformerclassification-model",
    "*",
    "training",
)
training_instance_type = "ml.m5.2xlarge"

# Retrieve the docker image
train_image_uri = image_uris.retrieve(
    region=aws_region,
    framework=None,
    model_id=train_model_id,
    model_version=train_model_version,
    image_scope=train_scope,
    instance_type=training_instance_type,
)

# Retrieve the training script
train_source_uri = script_uris.retrieve(
    model_id=train_model_id, model_version=train_model_version, script_scope=train_scope
)
# Retrieve the pre-trained model tarball to further fine-tune. In tabular case, however, the pre-trained model tarball is dummy and fine-tune means training from scratch.
train_model_uri = model_uris.retrieve(
    model_id=train_model_id, model_version=train_model_version, model_scope=train_scope
)

INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.


### 2.2. Set Training Parameters

---

Now that we are done with all the setup that is needed, we are ready to train our tabular algorithm. To begin, let us create a [``sageMaker.estimator.Estimator``](https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html) object. This estimator will launch the training job. 

There are two kinds of parameters that need to be set for training. The first one are the parameters for the training job. These include: (i) Training data path. This is S3 folder in which the input data is stored, (ii) Output path: This the s3 folder in which the training output is stored. (iii) Training instance type: This indicates the type of machine on which to run the training.

The second set of parameters are algorithm specific training hyper-parameters. 

---

In [37]:
# Sample training data is available in this bucket
data_bucket = "mypersonalprojectdata"
training_data_prefix = "ICR-Data/train"
validation_data_prefix = "ICR-Data/validation"

training_dataset_s3_path = f"s3://{data_bucket}/{training_data_prefix}"
validation_dataset_s3_path = f"s3://{data_bucket}/{validation_data_prefix}"


output_bucket = sess.default_bucket()
output_prefix = "tabular-training"

s3_output_location = f"s3://{output_bucket}/{output_prefix}/output"

---
For algorithm specific hyper-parameters, we start by fetching python dictionary of the training hyper-parameters that the algorithm accepts with their default values. This can then be overridden to custom values.

---

In [38]:
from sagemaker import hyperparameters

# Retrieve the default hyper-parameters for fine-tuning the model
hyperparameters = hyperparameters.retrieve_default(
    model_id=train_model_id, model_version=train_model_version
)

# [Optional] Override default hyperparameters with custom values
hyperparameters["n_epochs"] = "40"
print(hyperparameters)

{'n_epochs': '15', 'patience': '10', 'learning_rate': '0.001', 'batch_size': '256', 'input_dim': '32', 'n_blocks': '4', 'attn_dropout': '0.2', 'mlp_dropout': '0.1', 'frac_shared_embed': '0.25'}


# We move on to train with automatic model tuning  

We use a HyperparameterTuner object to interact with Amazon SageMaker hyperparameter tuning APIs.

In [31]:
from sagemaker.tuner import ContinuousParameter, CategoricalParameter, HyperparameterTuner

use_amt = True

hyperparameter_ranges = {
    "learning_rate": ContinuousParameter(0.001, 0.01, scaling_type="Auto"),
    "batch_size": CategoricalParameter([128, 256, 512]),
    "attn_dropout": ContinuousParameter(0.0, 0.8, scaling_type="Auto"),
    "mlp_dropout": ContinuousParameter(0.0, 0.8, scaling_type="Auto"),
}

---
We start by creating the estimator object with all the required assets and then launch the training job.

---

In [32]:
from sagemaker.estimator import Estimator
from sagemaker.utils import name_from_base

training_job_name = name_from_base(f"jumpstart-{train_model_id}-training")

# Create SageMaker Estimator instance
tabular_estimator = Estimator(
    role=aws_role,
    image_uri=train_image_uri,
    source_dir=train_source_uri,
    model_uri=train_model_uri,
    entry_point="transfer_learning.py",
    instance_count=1,
    instance_type=training_instance_type,
    max_run=360000,
    hyperparameters=hyperparameters,
    output_path=s3_output_location,
)

KeyboardInterrupt: 

In [ ]:
# tune hyper-parameters with instances in AWS
tuner = HyperparameterTuner(
    tabular_estimator,
    "f1_score",
    hyperparameter_ranges,
    [{"Name": "f1_score", "Regex": "metrics={'f1': (\\S+)}"}],
    max_jobs=10,  # increase the max_jobs to achieve better performance from hyperparameter tuning
    max_parallel_jobs=2,
    objective_type="Maximize",
    base_tuning_job_name=training_job_name,
)

tuner.fit({"training": training_dataset_s3_path, "validation": validation_dataset_s3_path}, logs=True)

In [ ]:
print(f'The best training job: {tuner.best_training_job()}')

The "best" hyperparameters can be seen on the AWS console. The following is from the best training job (003) with the highest `f1_score` of 0.8399999737739563:

- `attn_dropout`=0.07659012217308075
- `batch_size`=256
- `frac_shared_embed`="0.25"
- `input_dim`="32"
- `learning_rate`=0.0010093233223492368
- `mlp_dropout`=0.3041698649411079
- `n_blocks`="4"
- `n_epochs`="15"
- `patience`="10"
- `sagemaker_container_log_level`=20

We now deploy and run inference on the Trained Tabular Model for ICR validation data to store the predictions to use in other ensemble models.

---

The model will output the probability of the sample for each class in the model (2). We start by retrieving the jumpstart artifacts and deploy the `tuner` that we trained.

---

In [40]:
inference_instance_type = "ml.m5.2xlarge"

# Retrieve the inference docker container uri
deploy_image_uri = image_uris.retrieve(
    region=aws_region,
    framework=None,
    image_scope="inference",
    model_id=train_model_id,
    model_version=train_model_version,
    instance_type=inference_instance_type,
)
# Retrieve the inference script uri
deploy_source_uri = script_uris.retrieve(
    model_id=train_model_id, model_version=train_model_version, script_scope="inference"
)

endpoint_name = name_from_base(f"jumpstart-inference-{train_model_id}")

# Use estimator with best hyperparameters from the previous step to deploy to a SageMaker endpoint
predictor = tuner.deploy(
    initial_instance_count=1,
    instance_type=inference_instance_type,
    entry_point="inference.py",
    image_uri=deploy_image_uri,
    source_dir=deploy_source_uri,
    endpoint_name=endpoint_name,
)

INFO:sagemaker:Repacking model artifact (s3://sagemaker-us-west-1-378421839225/tabular-training/output/jumpstart-pytorch-tabtransformerclassif-2023-07-01-20-37-27-315/output/model.tar.gz), script artifact (s3://jumpstart-cache-prod-us-west-1/source-directory-tarballs/pytorch/inference/tabtransformerclassification/v1.0.2/sourcedir.tar.gz), and dependencies ([]) into single tar.gz file located at s3://sagemaker-us-west-1-378421839225/sagemaker-jumpstart-2023-07-01-20-43-59-578/model.tar.gz. This may take some time depending on model size...


KeyboardInterrupt: 

In [ ]:
# read the data
test_data = df_val #pd.read_csv(test_data_file_name, header=None)
test_data.columns = ["Target"] + [f"Feature_{i}" for i in range(1, test_data.shape[1])]

df_val

In [ ]:
newline, bold, unbold = "\n", "\033[1m", "\033[0m"

import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import log_loss
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

# read the data
test_data = df_val #pd.read_csv(test_data_file_name, header=None)
test_data.columns = ["Target"] + [f"Feature_{i}" for i in range(1, test_data.shape[1])]

num_examples, num_columns = test_data.shape
print(
    f"{bold}The test dataset contains {num_examples} examples and {num_columns} columns.{unbold}\n"
)

# prepare the ground truth target and predicting features to send into the endpoint.
ground_truth_label, features = test_data.iloc[:, :1], test_data.iloc[:, 1:]

print(f"{bold}The first 5 observations of the data: {unbold} \n")
test_data.head(5)

---
The following code queries the endpoint you have created to get the prediction for each test example. 
The `query_endpoint()` function returns an array-like of shape (num_examples, num_classes), where each row indicates
the probability of the example for each class in the model. The num_classes is 2 in above test data.
Next, the predicted class label is obtained by taking the class label with the maximum probability over others for each example. 

---

In [ ]:
content_type = "text/csv"


def query_endpoint(encoded_tabular_data):
    # endpoint_name = endpoint_name
    client = boto3.client("runtime.sagemaker")
    response = client.invoke_endpoint(
        EndpointName=endpoint_name, ContentType=content_type, Body=encoded_tabular_data
    )
    return response


def parse_response(query_response):
    model_predictions = json.loads(query_response["Body"].read())
    predicted_probabilities = model_predictions["probabilities"]
    return np.array(predicted_probabilities)


# split the test data into smaller size of batches to query the endpoint due to the large size of test data.
batch_size = 1500
predict_prob = []
for i in np.arange(0, num_examples, step=batch_size):
    query_response_batch = query_endpoint(
        features.iloc[i : (i + batch_size), :].to_csv(header=False, index=False).encode("utf-8")
    )
    predict_prob_batch = parse_response(query_response_batch)  # prediction probability per batch
    predict_prob.append(predict_prob_batch)


predict_prob = np.concatenate(predict_prob, axis=0)
predict_label = np.argmax(predict_prob, axis=1)

## 4. Evaluate the Prediction Results Returned from the Endpoint

---
We evaluate the predictions results returned from the endpoint by following two ways.

* Visualize the predictions results by plotting the confusion matrix.

* Measure the prediction results quantitatively.

---

In [ ]:
# Visualize the predictions results by plotting the confusion matrix.
conf_matrix = confusion_matrix(y_true=ground_truth_label.values, y_pred=predict_label)
fig, ax = plt.subplots(figsize=(7.5, 7.5))
ax.matshow(conf_matrix, cmap=plt.cm.Blues, alpha=0.3)
for i in range(conf_matrix.shape[0]):
    for j in range(conf_matrix.shape[1]):
        ax.text(x=j, y=i, s=conf_matrix[i, j], va="center", ha="center", size="xx-large")

plt.xlabel("Predictions", fontsize=18)
plt.ylabel("Actuals", fontsize=18)
plt.title("Confusion Matrix", fontsize=18)
plt.show()

In [ ]:
def bal_log_loss(p, y):
    ind0 = np.where(y==0)[0]
    ind1 = np.where(y==1)[0]
    
    N0 = len(ind0)
    N1 = len(ind1)
    
    y0 = (y==0).astype(int)
    y1 = y.astype(int)
    
    return (- np.sum(y0*np.log(p[:, 0]))/N0 - np.sum(y1*np.log(p[:, 1]))/N1) / 2


# Measure the prediction results quantitatively.
eval_accuracy = accuracy_score(ground_truth_label.values, predict_label)
eval_f1 = f1_score(ground_truth_label.values, predict_label)
eval_log_loss = bal_log_loss(predict_prob, np.squeeze(ground_truth_label.values, axis=(1,)))

print(
    f"{bold}Evaluation result on test data{unbold}:{newline}"
    f"{bold}{accuracy_score.__name__}{unbold}: {eval_accuracy}{newline}"
    f"{bold}F1 {unbold}: {eval_f1}{newline}"
    f"{bold}Log-Loss {unbold}: {eval_log_loss}{newline}"
)

---
Next, we delete the endpoint corresponding to the trained model.

---

In [41]:
# Delete the SageMaker endpoint and the attached resources
predictor.delete_model()
predictor.delete_endpoint()

INFO:sagemaker:Deleting model with name: sagemaker-jumpstart-2023-07-01-20-21-28-285
INFO:sagemaker:Deleting endpoint configuration with name: jumpstart-inference-pytorch-tabtransfor-2023-07-01-20-21-28-285
INFO:sagemaker:Deleting endpoint with name: jumpstart-inference-pytorch-tabtransfor-2023-07-01-20-21-28-285


# Predictions for Test Set Submission


------


Using the reported hyperparameters for *jumpstart-pytorch-ta-230701-1231-003-c17981ed*, we retrain the estimator with all of the training data. Since our training data is already fairly small, this is done to not waste the relatively training data.


------

In [ ]:
# change defaults to optimal parameters, increase number of epochs 
hyperparameters['n_epochs'] = "40"
hyperparameters['learning_rate'] = 0.0010093233223492368
hyperparameters['attn_dropout'] = 0.07659012217308075
hyperparameters['mlp_dropout'] = 0.3041698649411079

# Create SageMaker Estimator instance
tabular_estimator = Estimator(
    role=aws_role,
    image_uri=train_image_uri,
    source_dir=train_source_uri,
    model_uri=train_model_uri,
    entry_point="transfer_learning.py",
    instance_count=1,
    instance_type=training_instance_type,
    max_run=360000,
    hyperparameters=hyperparameters,
    output_path=s3_output_location,
)

# Launch a SageMaker Training job by passing s3 path of the training data
tabular_estimator.fit(
    {"training": training_dataset_s3_path}, logs=True, job_name=training_job_name
)